Having a created dataset generate a new sentences and using the theory learned in the markov models calculate the probability of a new sentence that uses words from the training dataset and unseen


In [13]:
import pandas as pd
import math

In [14]:
Training_dataset = ["The quick brown fox jumps over the lazy dog",
"The monkey eat the bananas",
"The lion sleeps while the crocodile hunts",
"Bears are omnivores and eat meat and vegetables",
"I like sentences about animals for my markov model exercise"]

In [15]:
def get_vocabulary_size(Training_dataset):
    number_of_states = []
    for sentence in Training_dataset:
        for word in sentence.split(" "):
            if word not in number_of_states:
                number_of_states.append(word)
    return number_of_states

In [28]:
def create_skeleton_transitions_matrix(Training_dataset):
    number_of_states = get_vocabulary_size(Training_dataset)
    dictionary_dataframe = {}
    for word in number_of_states:
        dictionary_dataframe[word] = 1
    dataframe_estados = pd.DataFrame(dictionary_dataframe, index=number_of_states)

    return dataframe_estados

In [24]:
def create_transition_matrix(dataset):
    possible_states = get_vocabulary_size(dataset)
    dataframe_estados = create_skeleton_transitions_matrix(dataset)
    transitions_values = dataframe_estados.to_dict()
    number_of_transitions = 0
    number_of_appearances_in_dataset_of_state = 0
    
    for state in range(len(possible_states)):
        state_origin = possible_states[state]
        for state in range(len(possible_states)):
            state_transition = possible_states[state]
            for sentence in dataset:
                if state_origin in sentence:
                    number_of_appearances_in_dataset_of_state +=1
                split_sentence = sentence.split(" ")
                for word in range(len(split_sentence)-1):
                    if split_sentence[word] == state_origin and split_sentence[word+1] == state_transition:
                        number_of_transitions +=1
            numerator = number_of_transitions+1
            denominator = number_of_appearances_in_dataset_of_state+len(possible_states)
            transition_probability = numerator/denominator
            transitions_values[state_origin][state_transition] = transition_probability
            number_of_appearances_in_dataset_of_state = 0
            number_of_transitions = 0
    final_matrix = pd.DataFrame(transitions_values)
    # check_working_ok = final_matrix['brown'].sum()
    # print(check_working_ok)

    return final_matrix

In [25]:
def initial_state_probability(sentence, training_dataset):
    number_of_initial_state_in_dataset = 0
    number_of_sentences_in_dataset = len(training_dataset)
    number_of_possible_states = len(get_vocabulary_size(training_dataset))
    splitted_sentence = sentence.split(" ")
    initial_word = splitted_sentence[0]
    for sentence in training_dataset:
        if sentence.startswith(initial_word):
            number_of_initial_state_in_dataset +=1
    numerator = number_of_initial_state_in_dataset+1
    denominator = number_of_sentences_in_dataset+number_of_possible_states
    probability_of_initial_state = numerator/denominator
        
    return probability_of_initial_state

In [26]:
def get_sentence_probability(dataset,sentence):
    possible_states = get_vocabulary_size(dataset)
    probability = initial_state_probability(sentence, dataset)
    transitions_matrix = create_transition_matrix(dataset)
    splitted_sentence = sentence.split(" ")
    for word in range(len(splitted_sentence)-1):
        if word != 0:
            if splitted_sentence[word] in possible_states and splitted_sentence[word-1] in possible_states:
                transition_probability = transitions_matrix[splitted_sentence[word]][splitted_sentence[word-1]]
            else:
                transition_probability = 1/len(possible_states)**2
            probability = probability * transition_probability
        else:
            pass

    #In order to see the number 
    # probability = probability * 1000000    
    return math.log(probability)

In [29]:
get_sentence_probability(Training_dataset,"The rat sleeps over the fox")

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'monkey', 'eat', 'bananas', 'lion', 'sleeps', 'while', 'crocodile', 'hunts', 'Bears', 'are', 'omnivores', 'and', 'meat', 'vegetables', 'I', 'like', 'sentences', 'about', 'animals', 'for', 'my', 'markov', 'model', 'exercise']


-23.34720150754469

In [21]:
get_sentence_probability(Training_dataset,"The Bears sleeps over the fox")

1.0


-16.413892310911088

In [22]:
get_sentence_probability(Training_dataset,"Me encantan los perros muy limpios")

1.0


-31.609646651458227